In [0]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

#from google.colab import files
#src = list(files.upload().values())[0]    # add a new lib from computer
#open('mylib.py','wb').write(src)
#import mylib

import inception
inception.download()

# conv katmanların ismini alıcak bir fonk
def conv_layer_names():
  model = inception.Inception()
  names = [op.name for op in model.graph.get_operations() if op.type == 'Conv2D']
  model.close()
  return names

# Şimdi bu isimleri bir listeye atayalım
conv_names = conv_layer_names()

# gradient ascent uygulayacağımız fonk
def optimize_image(conv_id=0, feature=0, iterations=30, show_progress=30):
  model = inception.Inception()
  resized_image = model.resized_image
  
# parametre ile gelen conv_id'ye göre conv alıp tensorünü alıyoruz
  conv_name = conv_names[conv_id]
  tensor = model.graph.get_tensor_by_name(conv_name + ":0")
  
# operatör ekleyebilmek için inception modelinin grafiğini default yapıyoruz
  with model.graph.as_default():
#loss fonksiyonu belirtilen feature için bütün tensor değerlerinin ortalaması
# o feature için loss değerini alalım
    loss = tf.reduce_mean(tensor[:, :, :, feature])
    
#resized image'a göre loss fonksiyonu için gradient alıcaz
#bunu tf.gradients ile yapıyoruz
  gradient = tf.gradients(loss, resized_image)
  
# Grafiği çalıştırmak için tf session açıyoruz  
  sess = tf.Session(graph=model.graph)
  
# rastgele resized_image boyutunda bir resim üretiyoruz(önce boyut alalım)
  image_shape = resized_image.get_shape()
  
# buna rastgele değerler atıyoruz np.random.uniform ile
# bu 128 değeri bize resim etrafında rastgele renkler vericek
# farklı değerler de atayabilirsin, bu şekilde görmesi daha kolay olucak
  image = np.random.uniform(size=image_shape) +  128.0
  
# optimizasyon 
  for i in range(iterations):
    feed_dict = {model.tensor_name_resized_image: image}
    [grad, loss_value] = sess.run([gradient, loss], feed_dict=feed_dict)

# gradient'i np.array'e sıkıştırıyoruz
# gradient artık bize, input resmini ne kadar değiştirirsek 
# feature'u maksimize ederizi gösteriyor
    grad = np.array(grad).squeeze()
    
# step size ile adım büyüklüğünü hesaplayacağız
# 1e-8 ile sıfırla bölünmesini engelliyoruz
# learning rate ile aynı şey aslında ama burda farklı bir şekilde kullanıyoruz
    step_size = 1.0 / (grad.std() + 1e-8)
# gradient ascent 
    image += step_size * grad
# np.clip ile resmin 0 ile 255 arasında olduğundan emin oluyoruz
    image = np.clip(image, 0.0, 255.0)
    
    if show_progress:
      print("Iteration: ", i)
      msg = "Gradient min: {0:9.6f}, max: {1:>9.6f}, stepsize: {2:>9.2f}"
      print(msg.format(grad.min(), grad.max(), step_size))
      print("Loss: ", loss_value)
      print()
      
  model.close()
  return image.squeeze()

# resmi normalleştiriyoruz(resmin değerlerini 0 ile 1 arasına sıkıştırıyoruz)
def normalize_image(x):
  x_min = x.min()
  x_max = x.max()
   
  x_norm = (x-x_min)/(x_max-x_min)
    
  return x_norm

# resimleri yazdıracak fonksiyonu yazalım
def plot_image(image):
  img_norm = normalize_image(image) #önce resmi normalleştiriyoruz
  plt.imshow(img_norm)
  plt.show()
  
# artık optimizeye başlayabiliriz

image = optimize_image(conv_id=10, feature=50, iterations=130)

plot_image(image)



Data already exists.
Iteration:  0
Gradient min: -0.000127, max:  0.000122, stepsize:  90423.76
Loss:  -1.8251309

Iteration:  1
Gradient min: -0.000052, max:  0.000064, stepsize: 231189.21
Loss:  -2.4000976

Iteration:  2
Gradient min: -0.000048, max:  0.000039, stepsize: 264003.32
Loss:  -2.3616502

Iteration:  3
Gradient min: -0.000045, max:  0.000046, stepsize: 270776.18
Loss:  -2.3103664

Iteration:  4
Gradient min: -0.000033, max:  0.000040, stepsize: 286408.96
Loss:  -2.245346

Iteration:  5
Gradient min: -0.000035, max:  0.000031, stepsize: 300172.23
Loss:  -2.1955256

Iteration:  6
Gradient min: -0.000036, max:  0.000047, stepsize: 310793.77
Loss:  -2.1389198

Iteration:  7
Gradient min: -0.000038, max:  0.000037, stepsize: 325209.35
Loss:  -2.0911376

Iteration:  8
Gradient min: -0.000038, max:  0.000039, stepsize: 322949.42
Loss:  -2.0499415

Iteration:  9
Gradient min: -0.000033, max:  0.000032, stepsize: 349654.51
Loss:  -2.01014

Iteration:  10
Gradient min: -0.000031, ma